In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
# read in the data
result = pd.read_csv("cleaned_result.csv")
result.head(5)

,index,id,name,brand,categories,manufacturer,rating,text,username,age
0,0,1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,5.0,This product so far has not disappointed. My c...,Adapter,52
1,1,1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,5.0,great for beginner or experienced person. Boug...,truman,51
2,2,1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,5.0,Inexpensive tablet for him to use and learn on...,DaveZ,23
3,3,1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,4.0,I've had my Fire HD 8 two weeks now and I love...,Shacks,25
4,4,1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,5.0,I bought this for my grand daughter when she c...,explore42,34


In [3]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
# make connection to the database
conn = Neo4jConnection(uri = "neo4j://localhost:7687",
                       user = "neo4j",              
                       pwd = "Hjm123456")

In [5]:
#clear database
def clear(conn):
    conn.query('match(n) detach delete n')

In [6]:
# Adds user nodes
def add_users():
    user_df = pd.read_csv("user.csv")
    
    query = '''
    UNWIND $rows AS row
    MERGE (u:User {username: row.username, age: row.age})
    '''
    return conn.query(query, parameters = {'rows': user_df.to_dict('records')})    

In [7]:
# Adds product nodes and the relation: user reviews product to the Neo4j graph.
def add_products():
    product_df = pd.read_csv("product.csv")
    
    query = '''
    UNWIND $rows AS row
    MERGE (p:Product {id: row.id, name: row.name, brand:row.brand, manufacturer:row.manufacturer})
    '''
    return conn.query(query, parameters = {'rows': product_df.to_dict('records')})

In [8]:
# Add relation between user and product: user reviews product.
def add_user_product_relation(dataframe):
    
    query = '''
    UNWIND $rows as row
    MATCH (u: User {username: row.username})
    MATCH (p:Product {id: row.id})
    MERGE (u)-[:Reviews {id:row.index, rating:row.rating, text:row.text}]->(p)
    '''
    return conn.query(query, parameters = {'rows': dataframe.to_dict('records')})

In [9]:
# Adds category nodes to the Neo4j graph.
def add_categories():
    
    buffer_category_lists = []
    unique_category = []
    
    category_lists = list(result["categories"])
        
    buffer_category_lists = [string.split(",") for string in category_lists]
    for i in buffer_category_lists:
        for j in i:
            unique_category.append(j)
    
    # get unique category list
    unique_category = list(set(unique_category))
    
    for category in unique_category:
        c = '"' + category +'"'
        query = ''' 
        create(c:Category {category: 
        ''' + c + '''})'''
        conn.query(query)

In [10]:
# Add the relationship between the product and category
def add_product_categories_relation(dataframe):
    for index,row in dataframe.iterrows():
        categories = list(row[4].split(","))
        product_id = "% s" % row[1]
        for c in categories:
            c = '"'+c+'"'
            query = '''
            MATCH (p:Product {id: '''+ product_id+'''})
            MATCH (c:Category {category:''' + c + '''})
            MERGE (p)-[:IN]->(c)
           '''
            conn.query(query)

In [11]:
#construct graph
clear(conn)
# set constrain to nodes so that they won't repeatly added 
conn.query('CREATE CONSTRAINT User IF NOT EXISTS ON (u:User) ASSERT u.username IS UNIQUE')
conn.query('CREATE CONSTRAINT Product IF NOT EXISTS ON (p:Product) ASSERT p.id IS UNIQUE')
conn.query('CREATE CONSTRAINT Category IF NOT EXISTS ON (c:Category) ASSERT c.category IS UNIQUE')

add_products()
print('product finished')
add_categories()
print('category finished')
add_users()
print('user finished')
add_user_product_relation(result)
print('review finished')
add_product_categories_relation(result)
print('in_category relation finished')

product finished
category finished
user finished
review finished
in_category relation finished


In [19]:
#build content-based filtering query
def cbf_query(username):
    username = '"' + username+ '"'
    query_string = '''MATCH(u:User{username:'''+ username +'''})-[:Reviews]->(p:Product)-[:IN]->(c:Category)<-[:IN]-(p2:Product)
WHERE NOT EXISTS ((u)-[:Reviews]->(p2))
WITH p, p2, COUNT(distinct c) AS intersection
MATCH (p)-[:IN]->(pc:Category)
WITH p, p2, intersection, COLLECT(pc.category) AS s1
MATCH (p2)-[:IN]->(p2c:Category)
WITH p, p2, s1, intersection, COLLECT(p2c.category) AS s2
WITH p, p2, intersection, s1+[x IN s2 WHERE NOT x IN s1] AS unionSet, s1, s2
RETURN p.id as Userproduct, p2.id as Recommendate, 
s1 as UserProductCategory, s2 as RecommendateProductCategory,
((1.0*intersection)/SIZE(unionSet)) AS jaccard ORDER BY jaccard DESC
limit 10'''
    
    recommand_list = conn.query(query_string)
    
    return recommand_list

In [20]:
print(cbf_query('John'))

[<Record Userproduct=72 Recommendate=79 UserProductCategory=['Computers & Tablets', 'Amazon Tablets', 'All Tablets', 'Tablets', 'Fire Tablets'] RecommendateProductCategory=['Computers & Tablets', 'Amazon Tablets', 'All Tablets', 'Tablets', 'Fire Tablets'] jaccard=1.0>, <Record Userproduct=83 Recommendate=87 UserProductCategory=['E-Readers', 'Kindle E-readers', 'Amazon Ereaders', 'Computers & Tablets', 'Amazon Book Reader', 'EBook Readers', 'E-Readers & Accessories'] RecommendateProductCategory=['E-Readers', 'Kindle E-readers', 'Amazon Ereaders', 'Computers & Tablets', 'Amazon Book Reader', 'EBook Readers', 'E-Readers & Accessories'] jaccard=1.0>, <Record Userproduct=18 Recommendate=79 UserProductCategory=['Amazon Tablets', 'All Tablets', 'Computers & Tablets', 'Fire Tablets', 'Tablets'] RecommendateProductCategory=['Computers & Tablets', 'Amazon Tablets', 'All Tablets', 'Tablets', 'Fire Tablets'] jaccard=1.0>, <Record Userproduct=6 Recommendate=79 UserProductCategory=['Amazon Tablets',

## Collaborative Filtering By Customer's Review

In [21]:
# Input a user name and return at most 25 products that score the highest based on the products the input user buy.
def getRecommandProductsForUser(user_name):
    recommand_list = conn.query('''match (u1:User{username:\"'''+ user_name +'''\"})-[r1:Reviews]->(p1:Product)<-[r2:Reviews]-(u2:User)-[r3:Reviews]->(p2:Product) return p2.name, collect(distinct p1.id) as product_id, id(u2) as partner_id, (count(r1) + count(r2) + count(r3)) / 3  as score order by score desc limit 25''')
    
    return [x[0] for x in recommand_list]

In [22]:
print(getRecommandProductsForUser("Kacy"))

['Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta', 'Echo (White),,,\r\nEcho (White),,,', 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Black', 'Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue,,,', 'All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta', 'Echo (White),,,\r\nEcho (White),,,', 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta', 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta', 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers,', 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta', 'Echo (White),,,\r\nEcho (White),,,', 'Echo (White),,,\r\nEcho (White),,,', 'Amazon Kindle Paperwhite - eBook reader - 4 GB - 6 monochrome Paperwhite - touchscreen - Wi-Fi - black,,,', 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special O

## Collaborative Filtering By Age

In [23]:
# giiven a username, and the age difference, return the recommend product list
def collaborative_filtering_by_age(username, difference):
    query_for_age = '''MATCH (u: User {username:\"'''+ username +'''\"})
                       RETURN u.age
                    '''
    # get the age of this user
    age = conn.query(query_for_age)[0][0]
    lower_bound_age = age - difference
    upper_bound_age = age + difference
    name_string = "\"" + username +"\""
    
    query_for_products_string_list = ["MATCH (u: User)-[:Reviews]->(p:Product)\n", 
                                      "WHERE u.username <>", name_string, "\n",
                                      "AND u.age > ", str(lower_bound_age), "\n",
                                      "AND u.age < ", str(upper_bound_age), "\n",
                                      "RETURN DISTINCT p"]
    
    query_for_products = "".join(query_for_products_string_list)
    
    recommend_list = conn.query(query_for_products)
    
    return [x[0] for x in recommend_list]

In [24]:
# get the products that bought by users around Kacy's age (± 1)
collaborative_filtering_by_age("Kacy", 1)

[<Node id=45708 labels=frozenset({'Product'}) properties={'name': 'Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue,,,', 'id': 13, 'brand': 'Amazon', 'manufacturer': 'Amazon'}>,
 <Node id=45696 labels=frozenset({'Product'}) properties={'name': 'All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta', 'id': 1, 'brand': 'Amazon', 'manufacturer': 'Amazon'}>,
 <Node id=45721 labels=frozenset({'Product'}) properties={'name': 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Black', 'id': 26, 'brand': 'Amazon', 'manufacturer': 'Amazon'}>,
 <Node id=45744 labels=frozenset({'Product'}) properties={'name': 'Amazon Tap Smart Assistant Alexaenabled (black) Brand New', 'id': 37, 'brand': 'Amazon', 'manufacturer': 'Amazon'}>,
 <Node id=45725 labels=frozenset({'Product'}) properties={'name': 'Echo (White),,,\r\nEcho (White),,,', 'id': 30, 'brand': 'Amazon', 'manufacturer': 'Amazon'}>,
 <Node id=45705 labels=frozenset({'Product'}) p

In [ ]:
# close connection
conn.close()